This is a modified version of a code for building a 3D CNN model used for lithology classification. 
The original code was made by Kurdistan Chawshin,  chawshinkurdistan@gmail.com, 2021

In [ ]:
import numpy as np 
seed = 7
np.random.seed(seed)
import os 
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

from tensorflow import keras
from tensorflow.keras import optimizers
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import time
t0 = time.time()

loading the data.

In [ ]:
aaaiTrainImages = np.load('../3DFixed/TrainImageSlicesFor3DPermeabilityEstimationFixed.npy')
print(aaaiTrainImages.shape)

aaaiTestImages = np.load('../3DFixed/TestImageSlicesFor3DPermeabilityEstimationFixed.npy')
print(aaaiTestImages.shape)

aaaiValidationImages= np.load('../3DFixed/ValidationImageSlicesFor3DPermeabilityEstimationFixed.npy')
print(aaaiValidationImages.shape)

afTrainPermeability = np.load('../3DFixed/TrainPermeabilityFor3DPermeabilityEstimationFixed.npy')
afTrainPermeability  = np.log(afTrainPermeability )
print(afTrainPermeability.shape)

afTestPermeability  = np.load('../3DFixed/TestPermeabilityFor3DPermeabilityEstimationFixed.npy')
afTestPermeability  = np.log(afTestPermeability )
print(afTestPermeability.shape)

afValidationPermeability  = np.load('../3DFixed/ValidationPermeabilityFor3DPermeabilityEstimationFixed.npy')
afValidationPermeability  = np.log(afValidationPermeability )
print(afValidationPermeability.shape)

print(afValidationPermeability [0])

Define batch size and number of epochs for Keras Tuner

In [3]:
batch_size = 32
epochs = 10

convert each 19 x 19 x 19 image of the train, validation and test set into a matrix of size 19 x 19 x 19 1 which is fed into the network. 1 is for number of channels. We are working with grayscale images. Therefore we have 1 channel. RGB images have 3 channels.

In [4]:
XPix, YPix,ZPix = aaaiTrainImages.shape[1], aaaiTrainImages.shape[2],aaaiTrainImages.shape[3]
aaaiTrainImages = aaaiTrainImages.reshape(-1,XPix,YPix,ZPix,1)
aaaiValidationImages = aaaiValidationImages.reshape(-1,XPix,YPix,ZPix,1)
aaaiTestImages = aaaiTestImages.reshape(-1,XPix,YPix,ZPix,1)


Print image shape for train, validation and test sets

In [ ]:
print ('Train Images shape with channel:', aaaiTrainImages.shape)
print ('Validation Images shape with channel:', aaaiValidationImages.shape)
print ('Test Images shape with channel:', aaaiTestImages.shape)
print ('Train permeability shape:', afTrainPermeability .shape)
print ('Validation Permeability shape:', afValidationPermeability .shape)
print ('Test Permeability  shape:', afTestPermeability .shape)


Rescale the pixel values between 0 and 1.

In [ ]:
aaaiTrainImages = aaaiTrainImages/255. 
aaaiValidationImages = aaaiValidationImages/255.
aaaiTestImages = aaaiTestImages/255.
print(np.min(aaaiTrainImages))

Import packages for CNN training

In [7]:
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Dense, Flatten, Dropout 
from sklearn.metrics import mean_squared_error 
from tensorflow.keras.layers import Conv3D, MaxPooling3D, ReLU, GlobalAveragePooling3D
from tensorflow.keras import activations
from tensorflow.keras.callbacks import EarlyStopping
from keras_tuner import Hyperband
from keras_tuner.engine.hyperparameters import HyperParameters

Define the model structure. Here we perform hyperparameter tuning useing Keras Tuner library. 
A range of values are defined for each of the considered hyperparameters. The algorithm will
consider different combinations and return the best configuration as the best performing model.

In [8]:
def ModelCreation(hp):
    Regressor = keras.Sequential()
    
    Regressor.add(keras.layers.Conv3D(
            filters= hp.Int('ConvFiltersInputLayer', min_value = 16, max_value=256,step=32),
            kernel_size=hp.Choice('Conv2Kernel', values=[3,5]), padding='same',input_shape =(XPix, YPix,ZPix,1)))

    Regressor.add(keras.layers.ReLU())
    Regressor.add(keras.layers.MaxPooling3D(
            pool_size=(2,2,2),padding='same'))
    
    for i in range(hp.Int('ConvBlocks', 1, 2, default=1)): 
        Regressor.add(keras.layers.Conv3D(
            filters= hp.Int(f'Conv{i}_Filter', min_value = 16, max_value=256, step=32),
            kernel_size= hp.Choice('Conv2Kernel', values=[3,5]), padding='same'))
    
        Regressor.add(keras.layers.ReLU())
        Regressor.add(keras.layers.MaxPooling3D(pool_size=(2,2,2),padding='same'))
        
        
        
    
 
    Regressor.add(keras.layers.Conv3D(
        filters= hp.Int('ConvFiltersInputLayer', min_value = 16, max_value=256,step=32),
        kernel_size=hp.Choice('Conv2Kernel', values=[3,5]), padding='same',input_shape =(XPix, YPix,ZPix, 1)))
        
 
    
    Regressor.add(keras.layers.ReLU()) 
    
   
    Regressor.add(keras.layers. GlobalAveragePooling3D())
    
    Regressor.add(keras.layers.Dense(1, activation='linear')) 
    
    
    Regressor.compile(loss="mean_absolute_error", optimizer=keras.optimizers.Adam(
        hp.Choice('LearningRate',[1e-2, 1e-3, 1e-4])))
    return Regressor

            
    

In [9]:
EarlyStop = EarlyStopping(monitor='val_loss', mode='min', patience=5)

Hyperband using keras-tuner

In [ ]:
Tuner = Hyperband(ModelCreation, objective= 'val_loss', max_epochs =epochs, executions_per_trial =1, seed=seed, directory=os.path.normpath('../3DFixed/NewArchitecture/codeOutputs/Permeability/TransportProp'), project_name='HyperUnRotValWholeDataManualSplit3DNewArchitectureFixedImages')

Tuner.search(aaaiTrainImages, afTrainPermeability , verbose=2, validation_data= (aaaiValidationImages, afValidationPermeability ), epochs=epochs, callbacks=[EarlyStop])

Tuner.search_space_summary()

Model = Tuner.get_best_models(1)[0]

print(Model.summary())

Tuner.results_summary()

print(Tuner.get_best_hyperparameters(1)[0].values)


Remove the 1 channel added to the images before hyperparameter tunning

In [13]:
XPix, YPix,ZPix = aaaiTrainImages.shape[1], aaaiTrainImages.shape[2],aaaiTrainImages.shape[3]
aaaiTrainImages = aaaiTrainImages.reshape(-1,XPix,YPix,ZPix)
aaaiValidationImages = aaaiValidationImages.reshape(-1,XPix,YPix,ZPix)
aaaiTestImages = aaaiTestImages.reshape(-1,XPix,YPix,ZPix)

In [ ]:
print ('Train Images shape without channel:', aaaiTrainImages.shape)
print ('Validation Images shape without channel:', aaaiValidationImages.shape)
print ('Test Images shape without channel:', aaaiTestImages.shape)
print ('Train Permeability shape:', afTrainPermeability .shape)
print ('Validation Permeability shape:', afValidationPermeability .shape)
print ('Test Permeability shape:', afTestPermeability .shape)

Image Augmentation: We rotate train images by three angles of 90, 180, and 270 degress. 
These rotated images together with the original images will be used for training. Next these images are also flipped horizontally.

In [5]:
aaaiTrainImagesRot90 = np.rot90(aaaiTrainImages, axes=(-2,-1))
print(aaaiTrainImagesRot90.shape)
afTrainPermeability90 = np.copy(afTrainPermeability )
print(afTrainPermeability90.shape)

aaaiTrainImagesRot180 = np.rot90(aaaiTrainImagesRot90, axes=(-2,-1))
print(aaaiTrainImagesRot180.shape)
afTrainPermeability180 = np.copy(afTrainPermeability )
print(afTrainPermeability180.shape)

aaaiTrainImagesRot270 = np.rot90(aaaiTrainImagesRot180, axes=(-2,-1))
print(aaaiTrainImagesRot270.shape)
afTrainPermeability270 = np.copy(afTrainPermeability )
print(afTrainPermeability270.shape)


aaaiTrainImagesHorFlip = np.fliplr(aaaiTrainImages)
afTrainPermeabilityHorFlip = np.copy(afTrainPermeability )
print(aaaiTrainImagesHorFlip.shape)
print(afTrainPermeabilityHorFlip.shape)


aaaiTrainImagesHorFlip90 = np.fliplr(aaaiTrainImagesRot90)
afTrainPermeabilityHorFlip90 = np.copy(afTrainPermeability )
print(aaaiTrainImagesHorFlip90.shape)
print(afTrainPermeabilityHorFlip90.shape)



aaaiTrainImagesHorFlip180 = np.fliplr(aaaiTrainImagesRot180)
afTrainPermeabilityHorFlip180 = np.copy(afTrainPermeability )
print(aaaiTrainImagesHorFlip180.shape)
print(afTrainPermeabilityHorFlip180.shape)

aaaiTrainImagesHorFlip270 = np.fliplr(aaaiTrainImagesRot270)
afTrainPermeabilityHorFlip270 = np.copy(afTrainPermeability )
print(aaaiTrainImagesHorFlip270.shape)
print(afTrainPermeabilityHorFlip270.shape)


aaaiTrainImages = np.concatenate((aaaiTrainImages, aaaiTrainImagesRot90, aaaiTrainImagesRot180, aaaiTrainImagesRot270, aaaiTrainImagesHorFlip,aaaiTrainImagesHorFlip90,aaaiTrainImagesHorFlip180,aaaiTrainImagesHorFlip270), axis=0)
afTrainPermeability  = np.concatenate((afTrainPermeability , afTrainPermeability90, afTrainPermeability180, afTrainPermeability270,afTrainPermeabilityHorFlip, afTrainPermeabilityHorFlip90,afTrainPermeabilityHorFlip180,afTrainPermeabilityHorFlip270), axis=0)

np.save('../3DFixed/NewArchitecture/afTrain3DPermeabilityAugmentationConcatenateOldArchitectureFixedImagesAugmentation.npy',afTrainPermeability)
np.savetxt('../3DFixed/NewArchitecture/afTrain3DPermeabilityAugmentationConcatenateOldArchitectureFixedImagesAugmentation.csv', afTrainPermeability )

print(aaaiTrainImages.shape)
print(afTrainPermeability .shape)

convert each 19 x 19 x 19 image of the train, validation and test set into a matrix of size 19 x 19 x 19 1 which is fed into the network. 1 is for number of channels. We are working with grayscale images. Therefore we have 1 channel. RGB images have 3 channels.

In [17]:
XPix, YPix,ZPix = aaaiTrainImages.shape[1], aaaiTrainImages.shape[2],aaaiTrainImages.shape[3]
aaaiTrainImages = aaaiTrainImages.reshape(-1,XPix,YPix,ZPix,1)
aaaiValidationImages = aaaiValidationImages.reshape(-1,XPix,YPix,ZPix,1)
aaaiTestImages = aaaiTestImages.reshape(-1,XPix,YPix,ZPix,1)

In [ ]:
print ('Train Images shape with channel:', aaaiTrainImages.shape)
print ('Validation Images shape with channel:', aaaiValidationImages.shape)
print ('Test Images shape with channel:', aaaiTestImages.shape)
print ('Train Permeability shape:', afTrainPermeability .shape)
print ('Validation Permeability shape:', afValidationPermeability .shape)
print ('Test Permeability shape:', afTestPermeability .shape)

The best returned model will train for more epochs.


In [19]:
Epochs = 300
EarlyStop = EarlyStopping(monitor='val_loss', mode='min', patience=30)

In [29]:
experiment = "../CheckPointTest/ThreeDNewArchitectureFixedAugmentation/"

ckpt_pathname = experiment + "/cp-{epoch:04d}.ckpt"
csv_filename = experiment + "/metricsThreeDNewArchitectureFixedAugmentation.csv"

Create a callback that saves the model's weights.

In [30]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=ckpt_pathname, verbose=1)
metrics_callback = tf.keras.callbacks.CSVLogger(csv_filename, append=True)

In [ ]:
BestRegressor = Model.fit(aaaiTrainImages, afTrainPermeability, validation_data = (aaaiValidationImages, afValidationPermeability), epochs=Epochs, callbacks =[EarlyStop, cp_callback,metrics_callback], verbose=2)

This loads the specified checkpoint you want

In [32]:
#Model = keras.models.load_model(experiment + "cp-0005.ckpt")

Fits the model after loading the ckechpoints an initial epoch you want to start with (eg:stopped after epoch 10, but initial epoch at 10, because it starts couting from 0)

In [33]:
#BestRegressor1 = Model.fit(aaaiTrainImages, afTrainPermeability , validation_data = (aaaiValidationImages, afValidationPermeability), epochs=Epochs, initial_epoch=5, callbacks =[EarlyStop, cp_callback,metrics_callback], verbose=2)

Evaluate the model using test set

In [ ]:
t1 = time.time()
print('Training took: ',(t1 - t0)/60,'minutes')



TestLoss  = Model.evaluate(aaaiTestImages, afTestPermeability, verbose=2)
print('Test loss:', TestLoss)

Plot the improvement in loss function for the training and validation sets.

In [ ]:


plt.plot(BestRegressor.history['loss'], color ='b')
plt.plot(BestRegressor.history['val_loss'], color = 'r')
#plt.title('model loss')
plt.ylabel('MAE')
plt.xlabel('Epochs')
plt.legend(['Training loss', 'Validation loss'], loc='upper right')
plt.savefig('3DCNNModelManualSplitNewArchFixedAugmentation2')


Predict test set and save the model and predicted results.

In [36]:
afPredictedPermeability  = Model.predict(aaaiTestImages)
np.savetxt('../codeOutputs/Permeability/TransportProp/PredictedPermeability300EpochsMAEWholeDataManualSplit3DNewArchitectureFixedAugmentation.csv', afPredictedPermeability)
save_model(Model, filepath= '../Models/PermeabilityEstimation/CNNModelForPermeabilityEstimation300EpochsMAEWholeDataManualSplit3DNewArchitectureFixedAugmentation', include_optimizer=True)